In [1]:
import sys
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
sys.path.append(".")
# os.environ["MAX_JOBS"] = "100"

import torch

In [2]:
torch.cuda.is_available()

True

In [3]:
from src.vlstm_v1.interface import testkernel, copykernel, mmkernelv1, mmkernelv2

INCLUDE: ['/home/max/miniconda3/envs/xlstmpt21cu121/lib/python3.11/site-packages/torch/include', '/home/max/miniconda3/envs/xlstmpt21cu121/lib/python3.11/site-packages/torch/include/torch/csrc/api/include', '/home/max/miniconda3/envs/xlstmpt21cu121/lib/python3.11/site-packages/torch/include/TH', '/home/max/miniconda3/envs/xlstmpt21cu121/lib/python3.11/site-packages/torch/include/THC', '/home/max/miniconda3/envs/xlstmpt21cu121/include']
/home/max/myrepos/vlstm_cuda/src
/usr/local/cuda-12.3/lib64:
/home/max/miniconda3/envs/xlstmpt21cu121/lib


Using /home/max/.cache/torch_extensions/py311_cu121 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/max/.cache/torch_extensions/py311_cu121/vlstm_v1/build.ninja...
Building extension module vlstm_v1...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/3] /home/max/miniconda3/envs/xlstmpt21cu121/bin/x86_64-conda-linux-gnu-c++ -MMD -MF interface.o.d -DTORCH_EXTENSION_NAME=vlstm_v1 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /home/max/miniconda3/envs/xlstmpt21cu121/lib/python3.11/site-packages/torch/include -isystem /home/max/miniconda3/envs/xlstmpt21cu121/lib/python3.11/site-packages/torch/include/torch/csrc/api/include -isystem /home/max/miniconda3/envs/xlstmpt21cu121/lib/python3.11/site-packages/torch/include/TH -isystem /home/max/miniconda3/envs/xlstmpt21cu121/lib/python3.11/site-packages/torch/include/THC -isystem /home/max/miniconda3/envs/xlstmpt21cu121/include -isystem /home/max/miniconda3/envs/xlstmpt21cu121/include/python3.11 -D_GLIBCXX_USE_CXX11_ABI=0 -fPIC -std=c++17 -c /home/max/myrepos/vlstm_cuda/src/vlstm_v1/interface.cc -o interface.o 
[2/3] /home/max/miniconda3/envs/xlstmpt21cu121/bin/nvcc  -ccbin /home/max/miniconda3

RuntimeError: Error building extension 'vlstm_v1'

In [ ]:
DTYPE = torch.bfloat16
DEVICE = torch.device("cuda:0")

In [ ]:
mat_A = torch.arange(4).reshape(2, 2).to(dtype=DTYPE, device=DEVICE)
mat_A

tensor([[0., 1.],
        [2., 3.]], device='cuda:0', dtype=torch.bfloat16)

In [ ]:
out = testkernel(mat_A)
out

Test kernel!


tensor([[0., 1.],
        [2., 3.]], device='cuda:0', dtype=torch.bfloat16)

In [ ]:
out = copykernel(mat_A.to(dtype=torch.float16))
out 

before kernel dispatch - float16!
rows: 2, cols: 2
blocksxy: 1-1, threads: 32-32
cidx: 0, ridx: 0, val: 0.000000
cidx: 1, ridx: 0, val: 2.000000
cidx: 0, ridx: 1, val: 1.000000
cidx: 1, ridx: 1, val: 3.000000


tensor([[0., 1.],
        [2., 3.]], device='cuda:0', dtype=torch.float16)

In [ ]:
warp_size = 32
am = torch.ones((1*1*warp_size, 1*1*warp_size), device=DEVICE, dtype=DTYPE)
bm = 2 * torch.ones((1*1*warp_size, 1*1*warp_size), device=DEVICE, dtype=DTYPE)
cm = am @ bm
cm, cm.shape

(tensor([[64., 64., 64.,  ..., 64., 64., 64.],
         [64., 64., 64.,  ..., 64., 64., 64.],
         [64., 64., 64.,  ..., 64., 64., 64.],
         ...,
         [64., 64., 64.,  ..., 64., 64., 64.],
         [64., 64., 64.,  ..., 64., 64., 64.],
         [64., 64., 64.,  ..., 64., 64., 64.]], device='cuda:0',
        dtype=torch.bfloat16),
 torch.Size([32, 32]))

In [ ]:
dt = torch.bfloat16
am = am.to(dtype=dt)
bm = bm.to(dtype=dt)
mmkernelv1(mat_A=am, mat_B=bm)

before kernel dispatch - bfloat16!
m: 32, n: 32, k: 32
blocksxy: 8-8, threads: 4-4


tensor([[64., 64., 64.,  ..., 64., 64., 64.],
        [64., 64., 64.,  ..., 64., 64., 64.],
        [64., 64., 64.,  ..., 64., 64., 64.],
        ...,
        [64., 64., 64.,  ..., 64., 64., 64.],
        [64., 64., 64.,  ..., 64., 64., 64.],
        [64., 64., 64.,  ..., 64., 64., 64.]], device='cuda:0',
       dtype=torch.bfloat16)

In [ ]:
S = 8 # sequence length
DH = 3 # hidden size

In [ ]:
matA = torch.ones((S, DH), device=DEVICE, dtype=DTYPE)
matB = torch.ones((DH, S), device=DEVICE, dtype=DTYPE)
matA.shape, matB.shape

(torch.Size([8, 3]), torch.Size([3, 8]))

In [ ]:
# pytorch
pt_out = matA @ matB
pt_out, pt_out.shape

(tensor([[3., 3., 3., 3., 3., 3., 3., 3.],
         [3., 3., 3., 3., 3., 3., 3., 3.],
         [3., 3., 3., 3., 3., 3., 3., 3.],
         [3., 3., 3., 3., 3., 3., 3., 3.],
         [3., 3., 3., 3., 3., 3., 3., 3.],
         [3., 3., 3., 3., 3., 3., 3., 3.],
         [3., 3., 3., 3., 3., 3., 3., 3.],
         [3., 3., 3., 3., 3., 3., 3., 3.]], device='cuda:0',
        dtype=torch.bfloat16),
 torch.Size([8, 8]))

In [ ]:
matA.is_contiguous(), matB.is_contiguous()

(True, True)

In [ ]:
cu_out = mmkernelv1(mat_A=matA, mat_B=matB)
cu_out, cu_out.shape

before kernel dispatch - bfloat16!
m: 8, n: 8, k: 3
blocksxy: 2-2, threads: 4-4


(tensor([[ 3., 67.,  3., 67.,  3., 67.,  3., 67.],
         [ 3., 67.,  3., 67.,  3., 67.,  3., 67.],
         [ 3., 67.,  3., 67.,  3., 67.,  3., 67.],
         [ 3., 67.,  3., 67.,  3., 67.,  3., 67.],
         [ 3., 67.,  3., 67.,  3., 67.,  3., 67.],
         [ 3., 67.,  3., 67.,  3., 67.,  3., 67.],
         [ 3., 67.,  3., 67.,  3., 67.,  3., 67.],
         [ 3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.]], device='cuda:0',
        dtype=torch.bfloat16),
 torch.Size([8, 8]))

In [ ]:
cu_out = mmkernelv2(mat_A=matA, mat_B=matB)
cu_out, cu_out.shape

before kernel dispatch - bfloat16!
m: 8, n: 8, k: 3
blocksxy: 2-2, threads: 4-4


(tensor([[ 3., 67.,  3., 67.,  3., 67.,  3., 67.],
         [ 3., 67.,  3., 67.,  3., 67.,  3., 67.],
         [ 3., 67.,  3., 67.,  3., 67.,  3., 67.],
         [ 3., 67.,  3., 67.,  3., 67.,  3., 67.],
         [ 3., 67.,  3., 67.,  3., 67.,  3., 67.],
         [ 3., 67.,  3., 67.,  3., 67.,  3., 67.],
         [ 3., 67.,  3., 67.,  3., 67.,  3., 67.],
         [ 3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.]], device='cuda:0',
        dtype=torch.bfloat16),
 torch.Size([8, 8]))

In [ ]:
# mat @ mat.T @ mat

### pytorch version

In [ ]:
H = 6 # hidden size
S = 5 # seq len
B = 1 # batch size
NH = 2 # num heads
DH = H // NH # dim per head
DTYPE = torch.bfloat16
DEVICE = torch.device("cuda:0")
assert H % NH == 0

In [ ]:
# create qkv, inputgates, forgetgates 
torch.manual_seed(0)
qs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
ks = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
vs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
ds = torch.rand((B, NH, S, S), device=DEVICE, dtype=DTYPE)

max_log_D, _ = torch.max(ds.view(B, NH, -1), dim=-1, keepdim=True)  # (B, NH, 1)
log_D_matrix_stabilized = ds - max_log_D.unsqueeze(-1)  # (B, NH, S, S) = (B, NH, S, S) - (B, NH, 1, 1)
D_matrix = torch.exp(log_D_matrix_stabilized)  # (B, NH, S, S)
mval = torch.exp(-max_log_D.unsqueeze(-1))